In [21]:
import tensorflow as tf
import keras as K
import os


import numpy as np
import pandas as pd
import re
import scipy as sp
from scipy.sparse import hstack, vstack
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
from sklearn.model_selection import KFold
import sklearn
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import BernoulliNB
from catboost import CatBoostClassifier
from sklearn import svm
import pickle
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from keras.callbacks import Callback
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_union
import sys
sys.executable

'/usr/bin/python3'

In [22]:
def save_variable_pickle(var, name, model = False):
    filename = name + '.pickle'
    if model:
        joblib.dump(var, filename = filename)
    else:  
        with open(filename, 'wb') as f:
            pickle.dump(var, f, pickle.HIGHEST_PROTOCOL)

def load_variable_pickle(name, model = False):
    filename = name
    if model:
        return joblib.load(filename = filename)
    else:
        with open(filename, 'rb') as f:
            return pickle.load(f)
        

In [23]:
def choose_system(os):
    if os == 'mac':
        train_path = '/Users/yichen/Desktop/toxic/input/train.csv'
        test_path = '/Users/yichen/Desktop/toxic/input/test.csv'
    elif os == 'linux':
        train_path = '/home/yichen/Desktop/toxic/input/train.csv'
        test_path = '/home/yichen/Desktop/toxic/input/test.csv'
    elif os == 'aws':
        train_path = 'train.csv'
        test_path = 'test.csv'
    
    train = pd.read_csv(train_path).fillna(' ')
    test = pd.read_csv(test_path).fillna(' ')
    return train, test
    
    

In [24]:
def check_accuracy(true, estimate):
    acc = accuracy_score(true, estimate)
    logloss = log_loss(true, estimate)
    auc = roc_auc_score(true, esimate)
    print("accuracy score: {}, logloss: {}, auc: {}".format(acc, logloss, auc))

In [25]:
train, test = choose_system('linux')
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
test.shape

(153164, 2)

In [26]:
repl = {
    "&lt;3": " good ", ":d": " good ",
    ":dd": " good ",
    ":p": " good ",
    "8)": " good ",
    ":-)": " good ",
    ":)": " good ",
    ";)": " good ",
    "(-:": " good ",
    "(:": " good ",
    "yay!": " good ",
    "yay": " good ",
    "yaay": " good ",
    "yaaay": " good ",
    "yaaaay": " good ",
    "yaaaaay": " good ",
    ":/": " bad ",
    ":&gt;": " sad ",
    ":')": " sad ",
    ":-(": " bad ",
    ":(": " bad ",
    ":s": " bad ",
    ":-s": " bad ",
    "&lt;3": " heart ",
    ":d": " smile ",
    ":p": " smile ",
    ":dd": " smile ",
    "8)": " smile ",
    ":-)": " smile ",
    ":)": " smile ",
    ";)": " smile ",
    "(-:": " smile ",
    "(:": " smile ",
    ":/": " worry ",
    ":&gt;": " angry ",
    ":')": " sad ",
    ":-(": " sad ",
    ":(": " sad ",
    ":s": " sad ",
    ":-s": " sad ",
    r"\br\b": "are",
    r"\bu\b": "you",
    r"\bhaha\b": "ha",
    r"\bhahaha\b": "ha",
    r"\bdon't\b": "do not",
    r"\bdoesn't\b": "does not",
    r"\bdidn't\b": "did not",
    r"\bhasn't\b": "has not",
    r"\bhaven't\b": "have not",
    r"\bhadn't\b": "had not",
    r"\bwon't\b": "will not",
    r"\bwouldn't\b": "would not",
    r"\bcan't\b": "can not",
    r"\bcannot\b": "can not",
    r"\bi'm\b": "i am",
    "m": "am",
    "r": "are",
    "u": "you",
    "haha": "ha",
    "hahaha": "ha",
    "don't": "do not",
    "doesn't": "does not",
    "didn't": "did not",
    "hasn't": "has not",
    "haven't": "have not",
    "hadn't": "had not",
    "won't": "will not",
    "wouldn't": "would not",
    "can't": "can not",
    "cannot": "can not",
    "i'm": "i am",
    "m": "am",
    "i'll" : "i will",
    "its" : "it is",
    "it's" : "it is",
    "'s" : " is",
    "that's" : "that is",
    "weren't" : "were not",
}

In [27]:
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
keys = [i for i in repl.keys()]

new_train_data = []
new_test_data = []
ltr = train["comment_text"].tolist()
lte = test["comment_text"].tolist()
for i in ltr:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_train_data.append(xx)
for i in lte:
    arr = str(i).split()
    xx = ""
    for j in arr:
        j = str(j).lower()
        if j[:4] == 'http' or j[:3] == 'www':
            continue
        if j in keys:
            # print("inn")
            j = repl[j]
        xx += j + " "
    new_test_data.append(xx)
train["new_comment_text"] = new_train_data
test["new_comment_text"] = new_test_data

trate = train["new_comment_text"].tolist()
tete = test["new_comment_text"].tolist()
for i, c in enumerate(trate):
    trate[i] = re.sub('[^a-zA-Z ?!]+', '', str(trate[i]).lower())
for i, c in enumerate(tete):
    tete[i] = re.sub('[^a-zA-Z ?!]+', '', tete[i])
train["comment_text"] = trate
test["comment_text"] = tete
del trate, tete
train.drop(["new_comment_text"], axis=1, inplace=True)
test.drop(["new_comment_text"], axis=1, inplace=True)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [28]:
all_text.shape

(312735,)

In [29]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))

In [30]:
import pickle
from sklearn.externals import joblib
test_features = load_variable_pickle('test_features_save.pickle')
train_features = load_variable_pickle('train_features_save.pickle')
y_fold_renew = load_variable_pickle('y_fold_valid.pickle')
#clf = load_variable_pickle('_logistic_model5.pickle')

In [31]:
print(test_features.shape, train_features.shape, y_fold_renew.shape)


(153164, 60000) (159571, 60000) (159571, 6)


In [32]:
from sklearn import svm

meta_valid_y = load_variable_pickle('meta_valid_y.pickle')
ada = AdaBoostClassifier(n_estimators=100)

rfc = RandomForestClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
nusvc = svm.NuSVC(nu=0.005,random_state = 1023, verbose = True)
#rfc= RandomForestClassifier(n_estimators= 50, random_state=11, verbose = True)
#submission[class_name] = classifier.predict_proba(test_features)[:, 1]
MNB = MultinomialNB()
gdc = GradientBoostingClassifier(n_estimators=400, 
 learning_rate = 0.1, max_depth=5, random_state=1023, verbose = True)
gbm_tune = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc', max_depth=12, min_data_in_leaf = 50, num_iterations = 100, 
                              learning_rate = 0.1)
                              #, device = 'gpu')
gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=500)# device = 'gpu')
etc = ExtraTreesClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
log_model = LogisticRegression(solver='sag')
svc_model = LinearSVC(random_state=1023)


model_set1 = []
model_set1.append(MNB)
model_set1.append(svc_model)
model_set1.append(log_model)
model_set1.append(etc)
#model_set1.append(ada)
model_set1.append(gbm_tune)
model_set1.append(rfc)

#model_set1.append(gdc)
model_set1.append(nusvc)

In [33]:
from sklearn import svm
model_set_shallow = []
log_model = LogisticRegression(solver='sag')
svc_model = LinearSVC(random_state=1023)
gbm_s = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc', max_depth = 3, min_data_in_leaf = 50, num_iterations = 100, 
                              learning_rate = 0.1)
MNB = MultinomialNB()
etc_s = ExtraTreesClassifier(n_estimators=100, min_samples_leaf= 100, random_state=1023, verbose = True)
rfc_s = RandomForestClassifier(n_estimators = 100, min_samples_leaf= 100, random_state=1023, verbose = True)
nusvc_s = svm.NuSVC(nu=0.005,random_state = 1023, verbose = True)
#cat_boost = CatBoostClassifier()

gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=500)
#ada = AdaBoostClassifier(n_estimators= 300)
model_set_shallow.append(log_model)
model_set_shallow.append(MNB)
model_set_shallow.append(svc_model)
model_set_shallow.append(etc_s)
model_set_shallow.append(rfc_s)
model_set_shallow.append(gbm_s)
#model_set_shallow.append(nusvc_s)

In [34]:
x_train_features, x_test_features, y_train_features, y_test_features = train_test_split(train_features, train,test_size = 0.5,shuffle = False)
print("x_train: {}, x_test: {}, y_train: {}, y_test: {}".format(x_train_features.shape, x_test_features.shape, y_train_features.shape, y_test_features.shape))


x_train: (79785, 60000), x_test: (79786, 60000), y_train: (79785, 8), y_test: (79786, 8)


In [35]:
def split_K_fold(train, target, K, model):
    acc = []
    kf = KFold(n_splits=K, random_state=1023)
    count = 0
    for train_index, test_index in kf.split(train):
        X_train, X_test = train[train_index], train[test_index]
        y_train, y_test = target[train_index], target[test_index]
        model.fit(X_train, y_train)
        if count == 0:
            pred = model.predict(X_test)
            #y_fold_valid = y_test
            #acc.append(roc_auc_score(pred, y_test))
        else:
            pred1 = model.predict(X_test)
            pred = np.concatenate((pred, pred1), axis = 0)
            #y_fold_valid = np.concatenate((y_fold_valid, y_test))
            #acc.append(roc_auc_score(pred1, y_test))
        count = count + 1
    #print("average roc_auc_score for model: {} is: {}".format(str(type(model)), np.mean(acc)))
    return pred

In [36]:
test_meta_gru= load_variable_pickle('meta_tune_nn_valid/test_meta_gru.pickle')
test_meta_lstm = load_variable_pickle('meta_tune_nn_valid/test_meta_lstm.pickle')
partB_meta_gru = load_variable_pickle('meta_tune_nn_valid/partB_meta_gru.pickle')
partB_meta_lstm = load_variable_pickle('meta_tune_nn_valid/partB_meta_lstm.pickle')
gru_y_pred = load_variable_pickle('meta_tune_nn_valid/gru_y_pred.pickle')
lstm_y_pred = load_variable_pickle('meta_tune_nn_valid/lstm_y_pred.pickle')
print(partB_meta_gru.shape)

(79786, 6)


In [37]:
from sklearn import svm
model_set_shallow = []
log_model = LogisticRegression(solver='sag')
svc_model = LinearSVC(random_state=1023)
gbm_s = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc', max_depth = 3, min_data_in_leaf = 50, num_iterations = 100, 
                              learning_rate = 0.1)
BNB = BernoulliNB()
MNB = MultinomialNB()
etc_s = ExtraTreesClassifier(n_estimators=100, min_samples_leaf= 100, random_state=1023, verbose = True)
rfc_s = RandomForestClassifier(n_estimators = 100, min_samples_leaf= 100, random_state=1023, verbose = True)
nusvc_s = svm.NuSVC(nu=0.005,random_state = 1023, verbose = True)
#cat_boost = CatBoostClassifier()
gdc = GradientBoostingClassifier(n_estimators=400, 
 learning_rate = 0.1, max_depth=5, random_state=1023, verbose = True)

gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=500)
ada = AdaBoostClassifier(n_estimators= 100)
#neigh2 = KNeighborsClassifier(n_neighbors=2)

#model_set_shallow.append(ada)
model_set_shallow.append(log_model)
#model_set_shallow.append(BNB)
model_set_shallow.append(MNB)
model_set_shallow.append(svc_model)
model_set_shallow.append(etc_s)
model_set_shallow.append(rfc_s)
model_set_shallow.append(gbm_s)
model_set_shallow.append(nusvc_s)
#model_set_shallow.append(neigh2)

In [38]:
meta_model_ada = AdaBoostClassifier(n_estimators= 300)
meta_model_gbm = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc',
                        num_leaves= 128,
                        learning_rate=0.1,
                        n_estimators=1500)

In [39]:
def meta_model_tune(models, test_data, partA_target, partA, partB, partB_meta_valid ,y_valid, meta_model):
    meta_roc_score = []
    
    
    class_counter = 0
    #submission = pd.DataFrame.from_dict({'id': test['id']})
    for class_name in class_names:
        os_load_file_path = '/home/yichen/Desktop/toxic/base_model_valid/'
        os_load_submission_path = '/home/yichen/Desktop/toxic/submission_valid/'
        os_load_submission_path = os_load_file_path + str(class_name) + str(type(meta_model))
        os_load_file_path = os_load_file_path + str(class_name) + '/'
        print(os_load_file_path)
        count = 0
        train_target = partA_target[class_name]
        for model in models:
            name = os_load_file_path +'base_lvl/'+ '_' + class_name + '_' + str(type(model)) + 'base_lvl' + '_' + '.pickle'
            if count == 0:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        partA_meta = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), partA_meta.shape))
                else:
                    partA_meta = split_K_fold(partA, train_target, 5, model)
                    with open(name, 'wb') as f:
                        pickle.dump(partA_meta, f, pickle.HIGHEST_PROTOCOL)
                count = count + 1
            else:
                if os.path.exists(name):
                    with open(name, 'rb') as f:
                        temp = pickle.load(f)
                    print("old prediction stored: {}, shape: {}".format(type(model), temp.shape))
                else:
                    temp = split_K_fold(partA, train_target, 5, model)
                    with open(name, 'wb') as f:
                        pickle.dump(temp, f, pickle.HIGHEST_PROTOCOL)
                partA_meta = np.column_stack((partA_meta, temp))
                count = count + 1
            print("partA.shape: ",partA_meta.shape)
        #add nn prediction
        
        
        count = 0                
        print("cv finished")
        for model in models:
            #print("count", count)
            partB_meta_name = os_load_file_path + 'partB_meta/' + class_name + '_' 'partB_meta' + '_'+ str(type(model)) + '.pickle'
            test_meta_name = os_load_file_path + 'meta_test/' + class_name + '_' + 'test_meta '+ '_' + str(type(model)) + '.pickle'
            
            if count == 0:
                if os.path.exists(partB_meta_name) and os.path.exists(test_meta_name):
                    with open(partB_meta_name, 'rb') as f:
                        partB_meta = pickle.load(f)
                    with open(test_meta_name, 'rb') as f:
                        test_meta = pickle.load(f)
                        
                    print("old prediction stored: {}, partB_meta: {}, test_meta: {}".format(type(model), partB_meta.shape, test_meta.shape))
                else:
                    model.fit(partA, train_target)
                    
                    partB_meta = model.predict(partB)
                    test_meta = model.predict(test_data)
                    
                    with open(partB_meta_name, 'wb') as f:
                        pickle.dump(partB_meta, f, pickle.HIGHEST_PROTOCOL)
                    with open(test_meta_name, 'wb') as f:
                        pickle.dump(test_meta, f, pickle.HIGHEST_PROTOCOL)
                        
                    print("model finish predication: {}".format(str(type(model))))
                count = count + 1
            else:
                if os.path.exists(partB_meta_name) and os.path.exists(test_meta_name):
                    with open(partB_meta_name, 'rb') as f:
                        temp_partB_meta  = pickle.load(f)
                    with open(test_meta_name, 'rb') as f:
                        temp_test_meta = pickle.load(f)
                        
                    print("old prediction stored: {}, partB_meta: {}, test_meta: {}".format(type(model), temp_partB_meta.shape, temp_test_meta.shape))
                else:
                    model.fit(partA, train_target)
                    temp_partB_meta = model.predict(partB)
                    temp_test_meta = model.predict(test_data)
                    
                    with open(partB_meta_name, 'wb') as f:
                        pickle.dump(temp_partB_meta, f, pickle.HIGHEST_PROTOCOL)
                    with open(test_meta_name, 'wb') as f:
                        pickle.dump(temp_test_meta, f, pickle.HIGHEST_PROTOCOL)
                        
                    print("model finish predication: {}".format(str(type(model))))
                    
                partB_meta = np.column_stack((partB_meta, temp_partB_meta))
                test_meta = np.column_stack((test_meta, temp_test_meta))
                #print("partB_meta: {}, test_meta: {}".format(partB_meta.shape, test_meta.shape))
                count = count + 1
        
        #meta_test = np.column_stack((meta_test, gru_meta_test[:, class_counter]))
        #meta_test = np.column_stack((meta_test, lstm_meta_test[:, class_counter]))
        partA_meta = np.column_stack((partA_meta, gru_y_pred[:, class_counter]))
        partA_meta = np.column_stack((partA_meta, lstm_y_pred[:, class_counter]))
        partB_meta = np.column_stack((partB_meta, partB_meta_gru[:, class_counter]))
        partB_meta = np.column_stack((partB_meta, partB_meta_lstm[:, class_counter]))
        #print("partA_meta with nn: {} and partB_meta with nn: {}".format(partA_meta.shape, partB_meta.shape))
        
        #save for whole nn meta
        if class_counter == 0:
            partA_meta_allclass = partA_meta
            partB_meta_allclass = partB_meta
        else:
            partA_meta_allclass = np.column_stack((partA_meta_allclass, partA_meta))
            partB_meta_allclass = np.column_stack((partB_meta_allclass, partB_meta))
        #we needs to take in nn prediction.
        #GRU and LSTM for now
        
        print("need this shape: ", partA_meta.shape)
        meta_train_valid = y_valid[:, class_counter]
        #this is for single class
        meta_model_name = '/home/yichen/Desktop/toxic/base_model_valid/' + class_name + '/' + 'meta_model_' + str(type(meta_model))
        if type(meta_model) == 'keras.models.Sequential':
            print("meta_model is nn")
            xt, xte, yt, yte = train_test_split(partA_meta, meta_train_valid, train_size=0.9)
            RocAuc = RocAucEvaluation(validation_data=(xte, yte), interval=1)
            meta_model.fit(xt, yt, epochs = 2, batch_size = 32, verbose = 1, callbacks=[RocAuc])
            partB_meta_pred = meta_model.predict(partB_meta)
            print("NN print partB_meta_pred should be (, ): ", partB_meta_pred.shape)
            ra_score = roc_auc_score(partB_meta_valid[class_name], partB_meta_pred)
            print("roc_auc_score for class: {} is {}".format(class_name, ra_score))
            meta_roc_score.append(ra_score)
        else:
            if os.path.exists(meta_model_name):
                print("meta_model same as last time")
                #meta_gbm = load_variable_pickle('meta_model_<class 'sklearn.ensemble.weight_boosting.AdaBoostClassifier'>.pickle')
            else:
                meta_model.fit(partA_meta, meta_train_valid)
                partB_meta_pred = meta_model.predict(partB_meta)
                #print("partB_meta_pred: {} and partB_meta_valid".format(partB_meta_pred.shape, partB_meta_valid[class_name]))
                ra_score = roc_auc_score(partB_meta_valid[class_name], partB_meta_pred)
                print("roc_auc_score for class: {} is {}".format(class_name, ra_score))
                meta_roc_score.append(ra_score)
                with open(meta_model_name, 'wb') as f:
                            pickle.dump(partB_meta_pred, f, pickle.HIGHEST_PROTOCOL)
                    
        #submission[class_name] = meta_y_test_pred
        class_counter = class_counter + 1
        print("round: ")
    save_variable_pickle(partA_meta_allclass, 'partA_meta_allclass')
    save_variable_pickle(partB_meta_allclass, 'partB_meta_allclass')
    print("the average roc_auc score is: {}".format(np.mean(meta_roc_score)))
    #submission.to_csv('submission_final.csv', index=False)

    return submission

In [30]:
meta_train_shallow = meta_model_tune(model_set_shallow, test_data = test_features,
                              partA_target = y_train_features, 
                              partA = x_train_features,
                              partB = x_test_features,
                              partB_meta_valid = y_test_features,
                              y_valid = meta_valid_y, 
                              meta_model = nn_model)
meta_train_shallow.shape

/home/yichen/Desktop/toxic/base_model_valid/toxic/
old prediction stored: <class 'sklearn.linear_model.logistic.LogisticRegression'>, shape: (79785,)
partA.shape:  (79785,)
old prediction stored: <class 'sklearn.naive_bayes.MultinomialNB'>, shape: (79785,)
partA.shape:  (79785, 2)
old prediction stored: <class 'sklearn.svm.classes.LinearSVC'>, shape: (79785,)
partA.shape:  (79785, 3)
old prediction stored: <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>, shape: (79785,)
partA.shape:  (79785, 4)
old prediction stored: <class 'sklearn.ensemble.forest.RandomForestClassifier'>, shape: (79785,)
partA.shape:  (79785, 5)
old prediction stored: <class 'lightgbm.sklearn.LGBMClassifier'>, shape: (79785,)
partA.shape:  (79785, 6)
old prediction stored: <class 'sklearn.svm.classes.NuSVC'>, shape: (79785,)
partA.shape:  (79785, 7)
cv finished
old prediction stored: <class 'sklearn.linear_model.logistic.LogisticRegression'>, partB_meta: (79786,), test_meta: (153164,)
old prediction stored: <c

/home/yichen/.local/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yichen/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yichen/.local/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/yichen/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check t

partA.shape:  (79785, 6)
[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]partA.shape:  (79785, 7)
cv finished
model finish predication: <class 'sklearn.linear_model.logistic.LogisticRegression'>
model finish predication: <class 'sklearn.naive_bayes.MultinomialNB'>
model finish predication: <class 'sklearn.svm.classes.LinearSVC'>


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   51.5s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   15.1s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   25.8s finished


model finish predication: <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>


...

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:  1.1min finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   14.9s finished
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   25.4s finished
/home/yichen/.local/lib/python3.5/site-packages/lightgbm/engine.py:99: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


model finish predication: <class 'sklearn.ensemble.forest.RandomForestClassifier'>


/home/yichen/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/yichen/.local/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


model finish predication: <class 'lightgbm.sklearn.LGBMClassifier'>
[LibSVM]model finish predication: <class 'sklearn.svm.classes.NuSVC'>
need this shape:  (79785, 9)
Epoch 1/1
79785/79785 [==============================] - 4s 56us/step - loss: 0.0331 - acc: 0.9928
roc_auc_score for class: identity_hate is 0.6683320552977111
round: 
the average roc_auc score is: 0.7606708539127661


NameError: name 'submission' is not defined

In [31]:
model_set_shallow.append(ada)
meta_model_tune(model_set_shallow, test_data = test_features,
                              partA_target = y_train_features, 
                              partA = x_train_features,
                              partB = x_test_features,
                              partB_meta_valid = y_test_features,
                              y_valid = meta_valid_y, 
                              meta_model = nn_model)

/home/yichen/Desktop/toxic/base_model_valid/toxic/
old prediction stored: <class 'sklearn.linear_model.logistic.LogisticRegression'>, shape: (79785,)
partA.shape:  (79785,)
old prediction stored: <class 'sklearn.naive_bayes.MultinomialNB'>, shape: (79785,)
partA.shape:  (79785, 2)
old prediction stored: <class 'sklearn.svm.classes.LinearSVC'>, shape: (79785,)
partA.shape:  (79785, 3)
old prediction stored: <class 'sklearn.ensemble.forest.ExtraTreesClassifier'>, shape: (79785,)
partA.shape:  (79785, 4)
old prediction stored: <class 'sklearn.ensemble.forest.RandomForestClassifier'>, shape: (79785,)
partA.shape:  (79785, 5)
old prediction stored: <class 'lightgbm.sklearn.LGBMClassifier'>, shape: (79785,)
partA.shape:  (79785, 6)
old prediction stored: <class 'sklearn.svm.classes.NuSVC'>, shape: (79785,)
partA.shape:  (79785, 7)


KeyboardInterrupt: 

In [40]:
from keras.layers import Dense, Activation
from keras.models import Sequential

In [41]:
nn_model = Sequential([Dense(100, input_shape = (9, ), activation='relu'),
                      Dense(100, activation='relu'),
                    Dense(6, activation='relu'),
                      Dense(1, activation='sigmoid')])
#nn_model.add(Dense(36, input_shape = (79785, ), activation='relu'))
#nn_model.add(Dense(24, activation='relu')
#nn_model.add(Dense(6, activation='sigmoid'))
nn_model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#nn_model.fit(, epochs = 3, batch_size = 32, verbose = 1)
nn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 100)               1000      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 606       
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 7         
Total params: 11,713
Trainable params: 11,713
Non-trainable params: 0
_________________________________________________________________


In [145]:
save_variable_pickle(y_test_features['toxic'], 'y_test_features_toxic')

In [ ]:
model_set1.append(neigh2)
model_set1.append(neigh4)
model_set1.append(neigh8)
model_set1.append(neigh16)
model_set1.append(neigh32)
meta_train2 = ready_for_leveltwo(model_set1, train_features, test_features, y_fold_validation, meta_model=gbm)

In [ ]:
meta_valid_y = load_variable_pickle('meta_valid_y.pickle')
meta_valid_y.shape, x_train_features.shape
y_test_features['toxic'].shape

In [ ]:
def cv_split(train, target, K, random_state ,model):
    acc = []
    row = train.shape[0]
    if row != target.shape[0]:
        raise ValueError("At least one array required as input")
    num_per_fold = np.int(np.ceil(train.shape[0]/K))
    x_train, x_fold, y_train, y_fold1 = train_test_split(train, target, test_size = num_per_fold, random_state = random_state)
    model.fit(x_train, y_train)
    pred = model.predict(x_fold)
    acc.append(roc_auc_score(y_fold1, pred))
    #print("accuracy is {}".format(accuracy_score(y_fold1, pred)))
    i = 1
    if train.size != row:
        while i < K:
            if i != K - 1:
                x_fold = train[num_per_fold * i: num_per_fold * (i + 1), :]
                y_fold = target[num_per_fold * i: num_per_fold * (i + 1)]
                x_train = vstack([train[0: num_per_fold * i, : ], train[num_per_fold * (i + 1): , :]])
                y_train = pd.concat([target[0 : num_per_fold * i], target[num_per_fold * (i + 1): ]])
                #print("x_train: {}, y_train: {}, x_fold: {}, y_fold: {}".format(x_train.shape, y_train.shape, x_fold.shape, y_fold.shape))
            elif i == K-1:
                x_fold = train[num_per_fold * i: , :]
                y_fold = target[num_per_fold * i: ]
                x_train = train[0:num_per_fold * i, :]
                y_train = target[0:num_per_fold * i]
                
            
            y_fold1 = pd.concat([y_fold1[:], y_fold[:]])
            model.fit(x_train, y_train)
            pred1 = model.predict(x_fold)
            acc.append(roc_auc_score(y_fold, pred1))
            #rint("accuracy is {}".format(accuracy_score(y_fold, pred1)))
            pred = np.concatenate((pred, pred1), axis = 0)
            i = i + 1
    print("average roc_auc_score is: {}  for model type: {} ".format(np.mean(acc), type(model)))
    return pred# pred_all_testep

In [ ]:
def single_class_cv_mutiple_time(model):
    result = np.arange(159571*6).reshape(159571,6)
    #all_test_result = np.arange(153164*6).reshape(153164,6)
    col = 0
    for class_name in class_names:
        train_target = train[class_name]
        test = test_features
        pred1, overall_test_result= cv_split(train = train_features, target = train_target, K = 5, random_state=1023, model=model)
        result[:, col] = pred1
        all_test_result[:, col] = overall_test_result
        #y_fold_result[:, col] = y_fold
        col = col + 1
        print("pred shape: {} and overall_test shape: {} ".format(pred1.shape, all_test_result.shape))
    
    return result, all_test_result


In [ ]:
def param_tune_all_class(model, train, train_t):
    score_auc = []
    for class_name in class_names:
        train_target = train_t[class_name]
        cv_score = np.mean(cross_val_score(model, train, train_target, cv=3, scoring='roc_auc'))
        scores_auc.append(np.abs(cv_score))
        print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))
    print('overall CV score for class is {}'.format(np.mean(score_auc)))

def param_tune_one_class(model, train, train_target):
    cv_score = np.mean(cross_val_score(model, train, train_target, cv=3, scoring='roc_auc'))
    print('auc CV score for class is {}'.format(np.abs(cv_score)))
    #print('overall CV score for class is {}'.format(np.mean(score_auc)))

In [ ]:
import sklearn

#print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
gdc_max_depth = [3, 5, 7, 10]
#for max_d in gdc_max_depth:
gdc = GradientBoostingClassifier(n_estimators=150,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=5, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

In [ ]:
rfc5 = RandomForestClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

In [ ]:
rfc5 = RandomForestClassifier(n_estimators=200, max_depth = 7, min_samples_leaf= 400, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

In [ ]:
gbm_tune = lgb.LGBMClassifier(objective = 'binary',
                        metric = 'auc', max_depth=12, min_data_in_leaf = 50, num_iterations = 100, 
                              learning_rate = 0.1
                              , device = 'gpu')
param_tune_one_class(gbm_tune, train_features, train['toxic'])

In [ ]:
rfc5 = RandomForestClassifier(n_estimators=300, 
                              max_depth = 7, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

In [ ]:
rfc5 = RandomForestClassifier(n_estimators= 400,max_depth = 7, random_state=1023, verbose = True)
param_tune_one_class(rfc5, train_features, train['toxic'])

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
etc5 = ExtraTreesClassifier(n_estimators=200, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=200, min_samples_leaf= 400, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=130, min_samples_leaf= 400, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=400, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
etc5 = ExtraTreesClassifier(n_estimators=600, min_samples_leaf= 200, random_state=1023, verbose = True)
param_tune_one_class(etc5, train_features, train['toxic'])

In [ ]:
gdc = GradientBoostingClassifier(n_estimators=200,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=5, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

In [ ]:
gdc = GradientBoostingClassifier(n_estimators=200,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=7, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

In [ ]:
gdc = GradientBoostingClassifier(n_estimators=200,# validation_fraction=0.2,
     learning_rate = 0.15, max_depth=10, random_state=1023, verbose = True)
param_tune_one_class(gdc, train_features, train['toxic'])

In [ ]:
param_tune_one_class(nusvc, train_features, train['toxic'])

In [ ]:
scores_auc = []
submission = pd.DataFrame.from_dict({'id': test['id']})
#classifier = LogisticRegression()
#for class_name in class_names:
train_target = train['toxic']
    #print(train_target.shape)
classifier = LogisticRegression(solver='sag')

    #cv_score = np.mean(cross_val_score(
     #   classifier, train_features, train_target, cv=3, scoring='neg_log_loss'))
    #scores_auc.append(np.abs(cv_score))
    #print('auc CV score for class {} is {}'.format(class_name, np.abs(cv_score)))

classifier.fit(train_features, train_target)
a = classifier.predict_proba(test_features)
print(a.shape)
a
submission[class_name] = a[:, 1]
    #save_varible_pickle(classifier, _logistic, model = True)
#print('Total auc CV score is {}'.format(np.mean(scores_auc)))

In [ ]:
submission.to_csv('submission.csv', index=False)

In [ ]:
def cv_split(train, target, K, random_state ,model):
    acc = []
    row = train.shape[0]
    if row != target.shape[0]:
        raise ValueError("At least one array required as input")
    num_per_fold = np.int(np.ceil(train.shape[0]/K))
    x_train, x_fold, y_train, y_fold1 = train_test_split(train, target, test_size = num_per_fold, random_state = random_state)
    model.fit(x_train, y_train)
    pred = model.predict(x_fold)
    acc.append(roc_auc_score(y_fold1, pred))
    #print("accuracy is {}".format(accuracy_score(y_fold1, pred)))
    i = 1
    if train.size != row:
        while i < K:
            if i != K - 1:
                x_fold = train[num_per_fold * i: num_per_fold * (i + 1), :]
                y_fold = target[num_per_fold * i: num_per_fold * (i + 1)]
                x_train = vstack([train[0: num_per_fold * i, : ], train[num_per_fold * (i + 1): , :]])
                y_train = pd.concat([target[0 : num_per_fold * i], target[num_per_fold * (i + 1): ]])
                #print("x_train: {}, y_train: {}, x_fold: {}, y_fold: {}".format(x_train.shape, y_train.shape, x_fold.shape, y_fold.shape))
            elif i == K-1:
                x_fold = train[num_per_fold * i: , :]
                y_fold = target[num_per_fold * i: ]
                x_train = train[0:num_per_fold * i, :]
                y_train = target[0:num_per_fold * i]
                
            
            y_fold1 = pd.concat([y_fold1[:], y_fold[:]])
            model.fit(x_train, y_train)
            pred1 = model.predict(x_fold)
            acc.append(roc_auc_score(y_fold, pred1))
            #rint("accuracy is {}".format(accuracy_score(y_fold, pred1)))
            pred = np.concatenate((pred, pred1), axis = 0)
            i = i + 1
    print("average roc_auc_score is: {}  for model type: {} ".format(np.mean(acc), type(model)))
    return pred# pred_all_testep